In [14]:
# Определяем список символов, которые нужно оставить
symbols = [str(digit) for digit in range(10)] + \
          [chr(ord('а') + i) for i in range(33)] + \
          ['.', ',', ':', ';', '-', ' ', '(', ')'] 

def filter_text_by_symbols(source_text, symbols):
    """
    Фильтрует текст, оставляя только символы из заданного списка.
    """
    return ''.join([char for char in source_text if char in symbols])


# Открываем файл, читаем содержимое, переводим в нижний регистр и фильтруем
try:
    with open('F_M_Dostoevskiy_Prestuplenie_i_nakazanie.txt', 'r', encoding='cp1251') as file:
        source_text = file.read()
    source_text = source_text.lower()
    # Удаляем ненужные символы
    filtered_text = filter_text_by_symbols(source_text, symbols)
    
    print(f"Длина исходного текста: {len(source_text)} символов")
    print(f"Длина обработанного текста: {len(filtered_text)} символов")
except FileNotFoundError:
    print("Файл не найден. Убедитесь, что он находится в той же директории и имеет правильное название.")


Длина исходного текста: 1195084 символов
Длина обработанного текста: 1160954 символов


In [15]:
import math


def calculate_information(probability):
    if probability == 0:
        return 0
    else:
        return -math.log2(probability)
    


def calculate_entropy(probability):
    if probability == 0:
        return 0
    else:
        return -probability * math.log2(probability)

def filter_text_by_symbols(source_text, symbols):
    return ''.join([char for char in source_text if char in symbols])

def create_character_frequency_dict(text):
    character_frequency = {}
    total_symbols_count = len(text)
    
    for symbol in symbols:
        count = text.count(symbol)
        probability = count / total_symbols_count if total_symbols_count > 0 else 0
        information = calculate_information(probability)
        entropy = calculate_entropy(probability)
        character_frequency[symbol] = {
            'code': ord(symbol),
            'occurrences': count,
            'probability': probability,
            'information': information,
            'entropy': entropy
        }
    
    return character_frequency

def calculate_total_entropy(character_frequency):
    """
    Вычисляет общую энтропию.
    """
    return sum(character_frequency[symbol]['entropy'] for symbol in character_frequency)

def validate_probabilities_sum(character_frequency):
    """
    Проверяет, что сумма вероятностей равна 1.
    """
    probabilities_sum = sum(character_frequency[symbol]['probability'] for symbol in character_frequency)
    return abs(probabilities_sum - 1) < 1e-6

def validate_symbols_count(character_frequency, source_text):
    """
    Проверяет, совпадает ли общее количество символов.
    """
    # Учитываем только символы, которые действительно встречаются в тексте
    total_symbols_count = sum(character_frequency[symbol]['occurrences'] for symbol in character_frequency if character_frequency[symbol]['occurrences'] > 0)
    return total_symbols_count == len(source_text)

# Генерация словаря частот символов
frequency_dict = create_character_frequency_dict(filtered_text)

# Печать информации по каждому символу
for key, value in sorted(frequency_dict.items()):
    print(f"{key}: {value}")

# Вычисление общей энтропии
total_entropy = calculate_total_entropy(frequency_dict)
print(f"\nОбщая энтропия источника: {total_entropy:.3f} бит")

# Проверка суммы вероятностей
if validate_probabilities_sum(frequency_dict):
    print("Вероятности в сумме дают 1 с точностью до ошибки округления.")
else:
    print("Вероятности в сумме не дают 1.")

# Проверка общего количества символов
if validate_symbols_count(frequency_dict, filtered_text):
    print(f"Общее количество символов ({len(filtered_text)}) совпадает с суммой количеств вхождений всех символов.")
else:
    print("Общее количество символов не совпадает с суммой количеств вхождений всех символов.")

 : {'code': 32, 'occurrences': 182286, 'probability': 0.15701397299117795, 'information': 2.6710351417063354, 'entropy': 0.4193898395983657}
(: {'code': 40, 'occurrences': 528, 'probability': 0.00045479838133121554, 'information': 11.102485259992957, 'entropy': 0.005049392324998477}
): {'code': 41, 'occurrences': 527, 'probability': 0.00045393702076051246, 'information': 11.105220227714195, 'entropy': 0.005041070585057961}
,: {'code': 44, 'occurrences': 26972, 'probability': 0.023232617313002926, 'information': 5.42770449702333, 'entropy': 0.12609978146740805}
-: {'code': 45, 'occurrences': 3552, 'probability': 0.003059552747137268, 'information': 8.352463513001304, 'entropy': 0.025554802686566937}
.: {'code': 46, 'occurrences': 9858, 'probability': 0.008491292505990762, 'information': 6.8798001136801785, 'entropy': 0.058418395148006896}
0: {'code': 48, 'occurrences': 108, 'probability': 9.302694163593044e-05, 'information': 13.391991877187941, 'entropy': 0.0012458160467480172}
1: {'co

In [16]:
import numpy as np

class ShannonFanoCode:
    def __init__(self, frequency_dict):
        # Инициализация с частотами символов
        self.symbols = [s for s in frequency_dict.keys()]
        self.ordered_symbols = sorted(
            [(s, frequency_dict[s]['occurrences']) for s in self.symbols],
            reverse=True, key=lambda x: x[1]
        )
        # Инициализируем коды
        self.codes = {s: '' for s in self.symbols}

    def create_codes(self, arr):
        # Если длина группы 1, дальнейшее деление невозможно
        if len(arr) <= 1:
            return

        # Извлекаем частоты
        occ = [x[1] for x in arr]

        # Поиск индекса для разделения на две группы с минимальной разницей
        total = sum(occ)
        cumulative = np.cumsum(occ)  # Накопленная сумма
        k = np.argmin(abs(cumulative - total / 2)) + 1  # Индекс для разделения

        # Добавляем '1' для первой группы и '0' для второй группы
        for i in range(len(arr)):
            if i < k:
                self.codes[arr[i][0]] += '1'
            else:
                self.codes[arr[i][0]] += '0'

        # Рекурсивное создание кодов для обеих групп
        self.create_codes(arr[:k])
        self.create_codes(arr[k:])

    def get_codes(self):
        return self.codes


In [17]:
shannon_fano = ShannonFanoCode(frequency_dict)
shannon_fano.create_codes(shannon_fano.ordered_symbols)
codes = shannon_fano.get_codes()
# Вывод кодов для символов с ненулевой частотой
print('Символ', 'Частота', 'Код', sep='\t')
for pair in shannon_fano.ordered_symbols:
    if pair[1] > 0:  # Исключаем символы с частотой 0
        print(f'{pair[0]}\t{pair[1]}\t{codes[pair[0]]}')

Символ	Частота	Код
 	182286	111
о	106730	110
е	80965	1011
а	73545	1010
и	62019	1001
н	60912	1000
т	59804	0111
с	50079	01101
в	43696	01100
л	42324	01011
р	39779	01010
к	30796	01001
д	29632	01000
м	29311	00111
у	27307	001101
,	26972	001100
п	25649	00101
ь	20554	001001
я	19749	001000
ч	16489	000111
г	16169	000110
б	16012	000101
ы	15449	0001001
з	14414	0001000
ж	10551	0000111
.	9858	0000110
й	9745	0000101
х	8125	0000100
ш	7437	0000011
ю	5418	00000101
-	3552	00000100
э	3201	00000011
щ	3039	000000101
ц	2782	000000100
;	1322	000000011
ф	1236	0000000101
:	978	0000000100
(	528	0000000011
)	527	00000000101
1	380	00000000100
8	295	00000000011
6	269	000000000101
ъ	223	000000000100
2	139	000000000011
5	132	0000000000101
4	130	0000000000100
7	121	0000000000011
3	120	0000000000010
0	108	0000000000001
9	96	00000000000001


In [10]:
import pandas as pd


total_length = len(filtered_text)
data = []
for symbol, occurrences in shannon_fano.ordered_symbols:
    # Вычисляем вероятность как отношение числа вхождений к общей длине текста
    probability = occurrences / total_length
    data.append([symbol, probability, codes[symbol]])

df = pd.DataFrame(data, columns=['Символ', 'Вероятность', 'Код'])

# Сортируем по вероятности в убывающем порядке
df_sorted = df.sort_values(by='Вероятность', ascending=False)

# Запись в файл Excel
df_sorted.to_excel('shannon_fano_codes.xlsx', index=False)

print("Таблица успешно записана в файл 'shannon_fano_codes.xlsx'.")

Таблица успешно записана в файл 'shannon_fano_codes.xlsx'.


In [11]:
def encode_text(text, codes):
    encoded_text = ''.join(codes[char] for char in text if char in codes)
    return encoded_text

# Кодируем текст
test_text = 'пример текстового сообщения.'
encoded_text = encode_text(test_text, codes)
print(f"Закодированный текст: {encoded_text}")
print(f"Длина закодированного текста: {len(encoded_text)} бит")

Закодированный текст: 0010101010100100111101101010111011110110100101101011111001100110000110110111011011101100001010000001011011100010010010000000110
Длина закодированного текста: 127 бит


In [12]:
def decode_text(text, codes):
    res = ''
    decode_dict = {v: k for k, v in codes.items()}  # Меняем местами коды и символы
    maxlen = max(map(len, decode_dict.keys()))
    
    while text:
        for i in range(1, maxlen + 1):  # Проверяем все возможные длины
            if text[:i] in decode_dict:
                res += decode_dict[text[:i]]
                text = text[i:]  # Убираем обработанную часть текста
                break
    return res

print(f'Декодированный текст: {decode_text(encoded_text, codes)}')

Декодированный текст: пример текстового сообщения.


In [13]:
def average_code_length(codes, frequency_dict):
    total_occurrences = sum(frequency_dict[char]['occurrences'] for char in codes)
    avg_length = sum(len(codes[char]) * frequency_dict[char]['occurrences'] for char in codes) / total_occurrences
    return avg_length

# Средняя длина кодового слова
avg_length = average_code_length(codes, frequency_dict)
print(f"Средняя длина кодового слова: {avg_length:.2f} бит")
print(f"Энтропия источника: {total_entropy:.2f} бит")

Средняя длина кодового слова: 4.55 бит
Энтропия источника: 4.52 бит


In [39]:
import heapq

class HuffmanNode:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None
    
    # Определяем поведение для heapq
    def __lt__(self, other):
        return self.freq < other.freq

# Построение дерева Хаффмана
def build_huffman_tree(frequency_dict):
    # Создаем кучу (heap) из символов и их частот
    heap = [HuffmanNode(char, frequency_dict[char]['occurrences']) for char in frequency_dict]
    heapq.heapify(heap)
    while len(heap) > 1:
        # Извлекаем два узла с наименьшей частотой
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        # Создаем объединённый узел
        merged = HuffmanNode(None, left.freq + right.freq)
        merged.left = left
        merged.right = right
        # Добавляем обратно в кучу
        heapq.heappush(heap, merged)
    # Возвращаем корень дерева
    return heap[0]

# Генерация кодов
def generate_huffman_codes(root, code='', codes=None):
    if codes is None:
        codes = {}
    if root:
        # Если лист
        if root.char is not None:
            codes[root.char] = code
        # Рекурсивно идём влево и вправо
        generate_huffman_codes(root.left, code + '0', codes)
        generate_huffman_codes(root.right, code + '1', codes)
    return codes

root = build_huffman_tree(frequency_dict)
codes = generate_huffman_codes(root)

# Вывод кодов
print("Символ", "Частота", "Код", sep="\t")
for char in frequency_dict:
    print(f"{char}\t{frequency_dict[char]['occurrences']}\t{codes[char]}")


Символ	Частота	Код
0	108	01110101111111
1	380	01110101000
2	139	0111010111110
3	120	0111010100110
4	130	0111010111000
5	132	0111010111001
6	269	011101011101
7	121	0111010100111
8	295	011101011110
9	96	011101011111101
а	73545	1000
б	16012	011110
в	43696	11100
г	16169	011111
д	29632	01000
е	80965	1010
ж	10551	1110100
з	14414	001001
и	62019	0110
й	9745	1001010
к	30796	01001
л	42324	10111
м	29311	00101
н	60912	0101
о	106730	000
п	25649	111011
р	39779	10011
с	50079	11110
т	59804	0011
у	27307	111111
ф	1236	1110101000
х	8125	0111011
ц	2782	111010101
ч	16489	100100
ш	7437	0010001
щ	3039	00100000
ъ	223	011101010010
ы	15449	011100
ь	20554	101101
э	3201	00100001
ю	5418	11101011
я	19749	101100
ѐ	0	011101011111100
.	9858	1001011
,	26972	111110
:	978	0111010101
;	1322	1110101001
-	3552	01110100
 	182286	110
(	528	01110101101
)	527	01110101100


In [37]:
data = []
for char in frequency_dict:
    occurrences = frequency_dict[char]['occurrences']
    probability = occurrences / total_length  # Преобразуем в вероятность
    code = codes.get(char, '')
    data.append([char, probability, code])

# Создаем DataFrame
df = pd.DataFrame(data, columns=['Символ', 'Вероятность', 'Код'])

# Сортируем по вероятности в убывающем порядке
df_sorted = df.sort_values(by='Вероятность', ascending=False)

# Запись в файл Excel
df_sorted.to_excel('huffman_codes_with_probabilities.xlsx', index=False)

print("Таблица успешно записана в файл 'huffman_codes_with_probabilities.xlsx'.")

Таблица успешно записана в файл 'huffman_codes_with_probabilities.xlsx'.


In [38]:
def calculate_average_code_length(codes, frequency_dict):
    total_length = 0
    total_freq = 0
    for char in codes:
        code_length = len(codes[char])
        freq = frequency_dict[char]['occurrences']
        total_length += code_length * freq
        total_freq += freq
    average_length = total_length / total_freq
    return average_length

def calculate_entropy(frequency_dict):
    total_symbols = sum(frequency_dict[char]['occurrences'] for char in frequency_dict)
    entropy = 0
    for char in frequency_dict:
        prob = frequency_dict[char]['occurrences'] / total_symbols
        entropy -= prob * math.log2(prob) if prob > 0 else 0
    return entropy

# Средняя длина кодового слова
average_length = calculate_average_code_length(codes, frequency_dict)
print(f"Средняя длина кодового слова: {average_length:.2f} бит")

# Энтропия источника
entropy = calculate_entropy(frequency_dict)
print(f"Энтропия источника: {entropy:.2f} бит")


Средняя длина кодового слова: 4.54 бит
Энтропия источника: 4.51 бит


In [42]:
huffman_codes = codes

In [43]:
huffman_codes

{'о': '000',
 'щ': '00100000',
 'э': '00100001',
 'ш': '0010001',
 'з': '001001',
 'м': '00101',
 'т': '0011',
 'д': '01000',
 'к': '01001',
 'н': '0101',
 'и': '0110',
 'ы': '011100',
 '-': '01110100',
 '1': '01110101000',
 'ъ': '011101010010',
 '3': '0111010100110',
 '7': '0111010100111',
 ':': '0111010101',
 ')': '01110101100',
 '(': '01110101101',
 '4': '0111010111000',
 '5': '0111010111001',
 '6': '011101011101',
 '8': '011101011110',
 '2': '0111010111110',
 'ѐ': '011101011111100',
 '9': '011101011111101',
 '0': '01110101111111',
 'х': '0111011',
 'б': '011110',
 'г': '011111',
 'а': '1000',
 'ч': '100100',
 'й': '1001010',
 '.': '1001011',
 'р': '10011',
 'е': '1010',
 'я': '101100',
 'ь': '101101',
 'л': '10111',
 ' ': '110',
 'в': '11100',
 'ж': '1110100',
 'ф': '1110101000',
 ';': '1110101001',
 'ц': '111010101',
 'ю': '11101011',
 'п': '111011',
 'с': '11110',
 ',': '111110',
 'у': '111111'}